In [ ]:
import pandas as pd

# Erläuterungen: https://www.arcgis.com/home/item.html?id=dd4580c810204019a7b8eb3e0b329dd6

# > 250 MB, braucht seine Zeit, das zu laden
df = pd.read_csv("https://opendata.arcgis.com/api/v3/datasets/dd4580c810204019a7b8eb3e0b329dd6_0/downloads/data?format=csv&spatialRefId=4326")

bak = df

In [ ]:
df.head(10)

In [ ]:
# Wegen korrigierter Daten führt einfaches Addieren zu leicht inkorrekten Daten...

df[ ["AnzahlFall", "AnzahlTodesfall"] ].sum()

In [ ]:
# NeuerFall: 
#   0: Fall ist in der Publikation für den aktuellen Tag und in der für den Vortag enthalten
#   1: Fall ist nur in der aktuellen Publikation enthalten
#  -1: Fall ist nur in der Publikation des Vortags enthalten

# damit ergibt sich: Anzahl Fälle der aktuellen Publikation als Summe(AnzahlFall), 
# wenn NeuerFall in (0,1); Delta zum Vortag als Summe(AnzahlFall) wenn NeuerFall in (-1,1)

# -> die neuen minus die korrigierten Fälle gibt die Erhöhung des heutigen Tages

alte = df[ (df["NeuerFall"] == 0) ]["AnzahlFall"].sum()
#print(alte)

neue = df[ (df["NeuerFall"] == 1) ]["AnzahlFall"].sum()
#print(neue)

korrektur = df[ (df["NeuerFall"] == -1) ]["AnzahlFall"].sum()
#print(korrektur)

print("Differenz zum Vortag: " + str(neue + korrektur))
print("Gesamt: " + str(neue + alte))

In [ ]:
# Und jetzt pro Bundesland

alte = df[ (df["NeuerFall"] == 0) ].groupby("Bundesland")["Bundesland", "AnzahlFall"].sum()
print(alte)

neue = df[ (df["NeuerFall"] == 1) ].groupby("Bundesland")["Bundesland", "AnzahlFall"].sum()
print(neue)

korrektur = df[ (df["NeuerFall"] == -1) ].groupby("Bundesland")["Bundesland", "AnzahlFall"].sum()
print(korrektur)

#print(neue + alte)


In [ ]:
# wieder geht es nicht so einfach... wenn Bundesländer keine Zahlen oder keine Korrektur melden, gibt es NaN...

neue.set_axis(["AnzahlFallHeute"], axis=1, inplace=True)
korrektur.set_axis(["AnzahlKorrekturHeute"], axis=1, inplace=True)

heute = pd.concat([neue, korrektur], axis=1, sort=False).fillna(0).astype(int)

gesamt = alte.copy()

gesamt["DiffZuVortag"] = heute["AnzahlFallHeute"] + heute["AnzahlKorrekturHeute"]
gesamt["AnzahlFallHeute"] = heute["AnzahlFallHeute"]

gesamt["AnzahlFall"] = gesamt[["AnzahlFall", "AnzahlFallHeute"]].sum(axis=1)
gesamt.drop(gesamt.columns[2], axis=1, inplace=True)

gesamt = gesamt.fillna(0).astype(int)
print(gesamt)